# Brain Tumor Prediction
* #### Setup
* #### Exploring Dataset
* #### Splitting Train and Validation
* #### Visualizing the Dataset
* #### Creating the Model
* #### Training the Model
* #### Visualizing the Model Performance

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import pathlib
import PIL

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy

## Exploring Dataset

In [ ]:
root_path = '../input/brian-tumor-dataset'
data_path = root_path + '/Brain Tumor Data Set/Brain Tumor Data Set'
data_dir = pathlib.Path(data_path)

In [ ]:
data = pd.read_csv(root_path + '/metadata.csv', index_col = 'Unnamed: 0')
data.head(10)

In [ ]:
tumor = list(data_dir.glob('Brain Tumor/*'))
PIL.Image.open(str(tumor[0]))

In [ ]:
healthy = list(data_dir.glob('Healthy/*'))
PIL.Image.open(str(healthy[0]))

## Splitting Training and Validation Dataset

In [ ]:
batch_size = 32
img_height = 150
img_width = 150
seed = 42

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
class_names

## Visualizing the Dataset

In [ ]:
plt.figure(figsize=(12, 12))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

## Creating the Model

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(len(class_names))
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
model.summary()

## Train the model

In [ ]:
epochs = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

## Visualizing the Model Performance

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))

epochs_range = range(epochs)

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Training and validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend()
plt.title('Training and validation Loss=')